In [2]:
import pydicom
import numpy as np
import os
import h5py

In [3]:
def save_3_slices_together(dir_path, volume_shape=(440,440,645), output_prefix='stacked file'):
    save_dir = dir_path    
    image_volume = np.zeros(volume_shape)
    i=0
    for filename in os.listdir(dir_path):
        f = os.path.join(dir_path, filename)
        # checking if it is a file
        if os.path.isfile(f):
            img = pydicom.read_file(f)
            img_array = img.pixel_array
            image_volume[:,:,i]=img_array
            i+=1      
    print("The shape of volume array is {}".format(image_volume.shape))
    
    image_volume[:,:,-1] = image_volume[:,:,-2]
    total_slices = int(image_volume.shape[2] / 3)
    print(total_slices)
    for idx in range(1, total_slices-1):
        data = image_volume[:,:,idx-1:idx+2]
        output_filename = output_prefix + str(idx) + ".h5"
        if idx == 1:
            print("The shape of volume array is {}".format(data.shape))
        
        with h5py.File(save_dir + "/" + output_filename, 'w') as outFile:
            dataset = outFile.create_dataset('data', data=data)
            dataset.attrs['idx'] = (idx-1, idx, idx+1)
            dataset.attrs['note'] = "Three contiguous slices. Prepared for 2.5-D Pix2Pix"            

In [ ]:
rootdir = 'C:/Users/deker/Downloads/Siemens Vision Quadra-20220714T071152Z-003/Siemens Vision Quadra/Subject_115-117'

for root, dirs, files in os.walk(rootdir):
    if len(dirs) == 0: # means we are reaching a dir with no subdir
        save_3_slices_together(dir_path=root)